In [95]:
import pandas as pd

In [78]:
df = pd.read_csv('data_small.csv')
print(df.shape)

(289551, 17)


In [79]:
df.head()

,id,campaignName,description,categories,duration,monetaryGoal,country,city,state,date_created,deadline_date,launched_date,state_changed_at,backers_count,usd_pledged,url,target
0,1123889576,mars-on-earth-an-art-residency,Help a fine art photographer continue her proj...,Space Exploration,27,1000,US,Boston,MA,2015-06-24,2015-10-23,2015-09-26,2015-10-23,53,1884,https://www.kickstarter.com/projects/cassandra...,successful
1,1724173143,vulcan-i-rocket-powered-by-3d-printed-engine,Team of undergraduates racing to be the first ...,Space Exploration,30,15000,US,San Diego,CA,2014-05-06,2015-05-21,2015-04-21,2015-05-21,465,21882,https://www.kickstarter.com/projects/105499101...,successful
2,707260502,starscraper-the-next-generation-of-suborbital-...,What if we built a rocket that is better than ...,Space Exploration,31,10000,US,Boston,MA,2014-11-29,2015-01-09,2014-12-09,2015-01-09,294,17176,https://www.kickstarter.com/projects/burpg/sta...,successful
3,497637964,students-building-a-near-space-balloon-with-li...,A group of high school students are building a...,Space Exploration,30,150,US,Mountain View,CA,2014-11-19,2015-11-26,2015-10-27,2015-11-26,45,970,https://www.kickstarter.com/projects/136362214...,successful
4,1546008758,earth-360,Re-inventing the way we look at our planet by ...,Space Exploration,30,7500,US,Fairfield,CT,2012-04-11,2012-09-21,2012-08-22,2012-09-21,28,7576,https://www.kickstarter.com/projects/211370922...,successful


## Encode Strings

In [80]:
def create_dictionary(data):
    temp_dict = {}
    for key in data.iteritems():
        if key[1] not in temp_dict.keys():
            temp_dict[key[1]] = len(temp_dict)
    return temp_dict

In [81]:
# create dictionaries for string variables
city_dict = create_dictionary(df['city'])
state_dict = create_dictionary(df['state'])
country_dict = create_dictionary(df['country'])
cat_dict = create_dictionary(df['categories'])

# map dictionaries to dataframe
df['categories'] = df['categories'].map(cat_dict)
df['country'] = df['country'].map(country_dict)
df['city'] = df['city'].map(city_dict)
df['state'] = df['state'].map(state_dict)
df['target'] = df['target'].map({'failed':0, 'successful':1})

## Train/Test Split

In [82]:
from sklearn.model_selection import cross_val_score, train_test_split

In [83]:
# pick features
target = 'target'
features = ['categories', 'duration'
            , 'monetaryGoal', 'country']

# train/test split
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=22)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((231640, 4), (57911, 4), (231640,), (57911,))

In [84]:
X_train.head()

,categories,duration,monetaryGoal,country
132205,3,20,3000,0
271533,61,30,2500,0
260566,151,32,3500,0
201906,110,30,1000,0
230423,124,30,2000,0


## Model

In [85]:
!pip install xgboost

You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [86]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

In [87]:
model = RandomForestClassifier()
model.fit(X_train,y_train)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [11]:
%%time

y_pred = model.predict(X_test)
len(set(y_pred))
score = accuracy_score(y_pred, y_test)
print(f'model 1 accuracy score {score}')

model 1 accuracy score 0.7272711574657664
CPU times: user 200 ms, sys: 0 ns, total: 200 ms
Wall time: 199 ms


In [92]:
import pickle
# pickle.dump(model, open('model_rf_wed.pkl', 'wb'))

In [88]:
model2 = LogisticRegression()
model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [95]:
y_pred = model.predict(X_test)
len(set(y_pred))
score = accuracy_score(y_pred, y_test)
print(f'model 1 accuracy score {score}')

model 1 accuracy score 0.7264422993904439


In [100]:
model3 = XGBClassifier()
model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [101]:
y_pred = model.predict(X_test)
len(set(y_pred))
score = accuracy_score(y_pred, y_test)
print(f'model 1 accuracy score {score}')

model 1 accuracy score 0.7274265683548894


## Hyperparameter Tuning

In [96]:
rfc = RandomForestClassifier()
xgb = XGBClassifier()
lr = LogisticRegression()

param_grid = {'n_jobs': [-1]
            , 'n_estimators':[1000]
            , 'max_depth': [15,17,20,22]
    
}

gs_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5, verbose=1)
gs_rfc.fit(X_train, y_train)
print('rfc best params:', gs_rfc.best_params_)
print('rfc scores:', gs_rfc.best_score_)

# tried 100, 500, 1000 estimators and 1000 was best

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed: 10.3min finished


rfc best params: {'max_depth': 17, 'n_estimators': 1000, 'n_jobs': -1}
rfc scores: 0.7471593852529788


In [ ]:
y_pred = gs_rfc.predict(X_test)
len(set(y_pred))
score = accuracy_score(y_pred, y_test)
print(f'gs_rfc accuracy score {score}')

In [97]:
rfc = RandomForestClassifier(n_jobs = -1, n_estimators=1000, max_depth=17)
rfc.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=17, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [93]:
y_pred = rfc.predict(X_test)
len(set(y_pred))
score = accuracy_score(y_pred, y_test)
print(f'tuned rfc accuracy score {score}')

tuned rfc accuracy score 0.7469738046312445


In [94]:
import pickle
pickle.dump(model, open('model_rf_thurs.pkl', 'wb'))

In [31]:
param_grid = {'solver': ['lbfgs', 'newton', 'liblinear' ]
              , ''
    
}
gs_xgb = GridSearchCV(estimator=xgb, param_grid=param_grid, cv= 5, verbose=1)
gs_xgb.fit(X_train, y_train)
print("xgb best params:", gs_rfc.best_params_)
print('xgb scores:', gs_rfc.best_score_)

KeyboardInterrupt: 

## Heroku Testing

In [66]:
import json
import requests

In [109]:
url = 'http://kickstarter-success.herokuapp.com/'
url2 = 'http://kickstarter-success.herokuapp.com/visualizations'


data = {
  "campaignName": 'string',
  "categories": 1,
  "description": 'string',
  "monetaryGoal": 1000,
  "duration": 30,
  "country": 3
}

data2 = {
  "campaignName": 'string',
  "categories": 1,
  "description": 'string',
  "monetaryGoal": 10000,
  "duration": 30,
  "country": 3,
  "user_id": 15
}

data = json.dumps(data)
data2 = json.dumps(data2)

In [110]:
send_request = requests.post(url, data)
print(send_request)

send_request2 = requests.post(url2, data2)
print(send_request)

<Response [200]>
<Response [200]>


In [111]:
print(send_request.json())
print(send_request2.json())

{'results': 0, 'custom_stats': {'raising_more_success': 118725, 'category_success': '0.4023', 'category_average': '141170.0625', 'average_duration': '32.8020', 'average_backers': '90.1549', 'average_over': '106212.4516'}}
{'graph1': 'https://jbti-kickstarter-success.s3.us-east-2.amazonaws.com/visualizations/visual1-15.html', 'graph3': 'https://jbti-kickstarter-success.s3.us-east-2.amazonaws.com/visualizations/visual3-15.html'}


In [75]:
df.head()

,id,campaignName,description,categories,duration,monetaryGoal,country,city,state,date_created,deadline_date,launched_date,state_changed_at,backers_count,usd_pledged,url,target
0,1123889576,mars-on-earth-an-art-residency,Help a fine art photographer continue her proj...,0,27,1000,0,0,0,2015-06-24,2015-10-23,2015-09-26,2015-10-23,53,1884,https://www.kickstarter.com/projects/cassandra...,1
1,1724173143,vulcan-i-rocket-powered-by-3d-printed-engine,Team of undergraduates racing to be the first ...,0,30,15000,0,1,1,2014-05-06,2015-05-21,2015-04-21,2015-05-21,465,21882,https://www.kickstarter.com/projects/105499101...,1
2,707260502,starscraper-the-next-generation-of-suborbital-...,What if we built a rocket that is better than ...,0,31,10000,0,0,0,2014-11-29,2015-01-09,2014-12-09,2015-01-09,294,17176,https://www.kickstarter.com/projects/burpg/sta...,1
3,497637964,students-building-a-near-space-balloon-with-li...,A group of high school students are building a...,0,30,150,0,2,1,2014-11-19,2015-11-26,2015-10-27,2015-11-26,45,970,https://www.kickstarter.com/projects/136362214...,1
4,1546008758,earth-360,Re-inventing the way we look at our planet by ...,0,30,7500,0,3,2,2012-04-11,2012-09-21,2012-08-22,2012-09-21,28,7576,https://www.kickstarter.com/projects/211370922...,1


In [ ]:
df_money = 